In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

inference_data:
    split: '?'

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-abs_area_estimation_error'

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 

## Set the high level name
group: "OCTA_FULLRES_Updated_Benchmark"

## Set the name of the dataset so we can automatically load the best models.
evaluate_dataset: "OCTA"

# subgroup: "Base"
# subgroup: "Sweep_Threshold"
subgroup: "Sweep_Temperature"
# subgroup: "Optimal_RAVE_Threshold"
# subgroup: "Optimal_RAVE_Temperature"

# Change this for debugging
# dataloader:
#     batch_size: 1

In [ ]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# For inference experiments, we don't add the date by default.
add_date = True 
# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    add_date=add_date
)

In [ ]:
len(inf_cfgs)

In [8]:
# Get the first half of the inference configs.
inf_cfgs = inf_cfgs[len(inf_cfgs)//2:]

## Running Jobs

In [9]:
from ese.analysis.run_inference import get_cal_stats

In [10]:
# from ese.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     show_examples=True,
#     gpu='2',
# )

In [ ]:
from ese.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    group="inference",
    base_cfg=updated_base_cfg,
    add_date=add_date,
    exp_cfg=experiment_cfg,
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    # available_gpus=['0', '1', '2', '3']
    available_gpus=['4', '5', '6', '7']
)